# 日本取引所グループ ニュース分析チャレンジ


- [data](https://signate.jp/competitions/443/data)
- [フォーラム](https://signate.jp/competitions/443/discussions)
- [チュートリアル](https://japanexchangegroup.github.io/J-Quants-Tutorial/#_%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9%E3%81%A7%E3%83%9D%E3%83%BC%E3%83%88%E3%83%95%E3%82%A9%E3%83%AA%E3%82%AA%E3%82%92%E6%A7%8B%E7%AF%89%E3%81%97%E3%82%88%E3%81%86)
- [github](https://github.com/JapanExchangeGroup/J-Quants-Tutorial/tree/main/handson/)

In [ ]:
!pip install TA-Lib
!pip install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import sklearn
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
from backtest.backtest import Backtest
import talib as ta

# 表示用の設定を変更します
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120


def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)

SEED = 42
fix_seed(SEED)

In [ ]:
from time import time

def decorate(s: str, decoration=None):
    if decoration is None:
        decoration = '★' * 20

    return ' '.join([decoration, str(s), decoration])

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        if self.verbose is None:
            return
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)
import inspect

def param_to_name(params: dict, key_sep='_', key_value_sep='=') -> str:
    """
    dict を `key=value` で連結した string に変換します.
    Args:
        params:
        key_sep:
            key 同士を連結する際に使う文字列.
        key_value_sep:
            それぞれの key / value を連結するのに使う文字列.
            `"="` が指定されると例えば { 'foo': 10 } は `"foo=10"` に変換されます.
    Returns:
        文字列化した dict
    """
    sorted_params = sorted(params.items())
    return key_sep.join(map(lambda x: key_value_sep.join(map(str, x)), sorted_params))


def cachable(function):
    attr_name = '__cachefile__'
    def wrapper(*args, **kwrgs):
        force = kwrgs.pop('force', False)
        call_args = inspect.getcallargs(function, *args, **kwrgs)

        arg_name = param_to_name(call_args)
        name = attr_name + arg_name

        use_cache = hasattr(function, name) and not force

        if use_cache:
            cache_object = getattr(function, name)
        else:
            print('run')
            cache_object = function(*args, **kwrgs)
            setattr(function, name, cache_object)

        return cache_object

    return wrapper
@cachable
def read_csv(name):

    if '.csv' not in name:
        name = name + '.csv'

    return pd.read_csv(name)

# Dataの読み込み

メモ: 最新のデータまで更新したものより、最初に配布されたstock_price_org.csvを用いたほうがシャープレシオが良かったのでこっちを作用

In [ ]:
dataset_dir = "./data"

inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    #"stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price_org.csv.gz",
}

## データを理解する

[公式の説明](https://japanexchangegroup.github.io/J-Quants-Tutorial/#anchor-3.4)  
基本的には、2016/1/1〜2020/12/31のデータになっている。  
ファンダメンタルチャレンジでも用いられたのは以下 (☆がメインだと思う)
- 銘柄情報のリスト
- ☆株価のヒストリカル
- ☆ファンダメンタル
- 各日付からN=5,10,20の間の最高値と最安値  

newsチャレンジで追加されたデータ
- ☆日経電子版の見出しとメタデータ
- 日経電子版のメタデータの記号の意味を書いたcsvがいくつか
- disclosureItems (株式分割(positive)や災害に起因する損害又は業務遂行の過程で生じた損害(negative)が含まれている)

☆の情報の中身から確認する

In [ ]:
# 銘柄情報読み込み
df_stock_list = read_csv(inputs["stock_list"])
# 問題2のユニバース (投資対象の条件を満たす銘柄群) 取得
codes = df_stock_list.loc[
    df_stock_list.loc[:, "universe_comp2"] == True, "Local Code"
].unique()
# 価格情報読み込み、インデックス作成
df_price = pd.read_csv(inputs["stock_price"]).set_index("EndOfDayQuote Date")
# 日付型に変換
df_price.index = pd.to_datetime(df_price.index, format="%Y-%m-%d")
# 並び替え
df_price = df_price.reset_index().sort_values(['Local Code', 'EndOfDayQuote Date']).set_index('EndOfDayQuote Date')

In [ ]:
df_price.reset_index().sort_values(['EndOfDayQuote Date'])

銘柄の情報リスト  

- prediction_target: fundamental challangeの残骸なので無視  
- universe_comp2: これがnews チャンレンジの対象銘柄  
- IssuedShareEquityQuote IssuedShare: 発行株式数なので重要  
その他は業種や規模感を表すもの

In [ ]:
df_price.tail().T

株のヒストリカルデータ  

個人的に重要だと思う順番  
- EndOfDayQuote ExchangeOfficialClose: 基本はこれを株価とする(取引が行われないときは前日の終値になる) (取引所公式終値。最終の特別気配または最終気配を含む終値)
- EndOfDayQuote Volume: その日の取引高
- EndOfDayQuote CumulativeAdjustmentFacto: 累積調整係数 最後の日付が1になるように調整されている(ただしデータリークになるので基本は使わないほうが良さそう)
> - 調整前株価 = 調整済株価 * 累積調整係数  
> - 調整前出来高 =  調整済出来高 / 累積調整係数  
> - つまり2だと1:2で株式分割されたということ
- EndOfDayQuote PercentChangeFromPreviousClose: 騰落率(前回終値からの直近約定値の上昇率または下落率)



日経新聞によるニュースデータ

- headline: 見出し
- keywords: キーワード
- classifications: 分類 (これの意味は他のcsvで与えられる)

# 全て0の日を除外

In [ ]:
# 2020/10/1 local codeが2つぶんが0だったので除外
df_price = df_price[df_price['EndOfDayQuote ExchangeOfficialClose']!=0.0]
df_price[df_price['EndOfDayQuote ExchangeOfficialClose']==0.0]

# 特徴ベクトルの作成



## sotck priceでの特徴量の作成

まずは対象となる銘柄と日付のfilter  
2020/1/1 〜2020/12/30を予測対象に


In [ ]:
start_dt = pd.Timestamp("2016-02-01")
# 投資対象日の前週金曜日時点で予測を出力するため、予測出力用の日付を設定します。
pred_start_dt = pd.Timestamp(start_dt) - pd.Timedelta("3D")
# 特徴量の生成に必要な日数をバッファとして設定
n = 30
data_start_dt = pred_start_dt - pd.offsets.BDay(n)
# 日付で絞り込み
filter_date = df_price.index >= data_start_dt
# 銘柄をユニバースで絞り込み
filter_universe = df_price.loc[:, "Local Code"].isin(codes)
# 絞り込み実施
df_price = df_price.loc[filter_date & filter_universe]

leakが起きそうな'EndOfDayQuote CumulativeAdjustmentFactor'と数値データ出ないものは取り除く

In [ ]:
def create_numeric_feature(input_df):
    all_columns = set(input_df.keys())
    except_columns = set(['EndOfDayQuote CumulativeAdjustmentFactor', 'EndOfDayQuote PreviousCloseDate', 'EndOfDayQuote PreviousExchangeOfficialCloseDate'])
    use_columns = all_columns - except_columns
    return input_df[use_columns].copy()

In [ ]:
def create_talib_feature(input_df):
    out_df = []
    target_label = 'EndOfDayQuote ExchangeOfficialClose'
    for target_code in tqdm(input_df['Local Code'].unique()):
        ret = {}
        df = input_df.loc[input_df.loc[:, 'Local Code']==target_code, target_label]
        prices = np.array(df, dtype='f8')
    
        ret['rsi7'] = ta.RSI(prices, timeperiod=7)
        ret['sma14'] = ta.SMA(prices, timeperiod=14)
        ret['sma7'] = ta.SMA(prices, timeperiod=7)
        ret['bb_up'], ret['bb_mid'], ret['bb_low'] = ta.BBANDS(prices, timeperiod=14)
        ret['mom'] = ta.MOM(prices, timeperiod=14)
        #ret['macd'], ret['macdsignal'], ret['macdhist'] = ta.MACD(prices, fastperiod=12, slowperiod=26, signalperiod=9)
        #ret['macd_macdsignal'] = ret['macd'] - ret['macdsignal']
        
        tmp_df = pd.DataFrame(ret, index=df.index)
        ret['sma7_diff'] = tmp_df['sma7'].diff()
        ret['sma14_diff'] = tmp_df['sma14'].diff()
        ret['sma_diff'] =tmp_df['sma7'] -tmp_df['sma14']
        ret['bb_diff'] = tmp_df['bb_mid'] - tmp_df['bb_up']
        #ret['macd_macdsignal_diff'] = tmp_df['macd_macdsignal'].diff()
        code_df = pd.DataFrame(ret, index=df.index)
        out_df.append(code_df)
    return pd.concat(out_df, axis=0)

In [ ]:
def create_return_feature(input_df):
    out_df = []
    target_label = 'EndOfDayQuote ExchangeOfficialClose'
    for target_code in tqdm(input_df['Local Code'].unique()):
        ret = {}
        df = input_df.loc[input_df.loc[:, 'Local Code'] == target_code, target_label]
        ret['retrun1'] = input_df[input_df['Local Code'] == target_code][target_label].pct_change(periods=1)
        ret['retrun7'] = input_df[input_df['Local Code'] == target_code][target_label].pct_change(periods=7)
        ret['retrun14'] = input_df[input_df['Local Code'] == target_code][target_label].pct_change(periods=14)
        code_df = pd.DataFrame(ret, index=df.index)
        out_df.append(code_df)
    return pd.concat(out_df, axis=0)

In [ ]:
def to_feature(input_df):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        create_numeric_feature,
        create_talib_feature,
        create_return_feature
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
    return out_df

In [ ]:
X = to_feature(df_price)

In [ ]:
X.head(30).T

# 予測対象の設定

(6日後のclose - 1日後のopen) / 1日後のopen

In [ ]:
def to_target(input_df):
    df_copy = input_df.copy()
    out_df = []
    end_label = 'EndOfDayQuote ExchangeOfficialClose'
    start_label = 'EndOfDayQuote Open'
    # open が存在しない場合の0割を回避
    df_copy.loc[df_copy['EndOfDayQuote Open'] == 0, 'EndOfDayQuote Open'] =  df_copy.loc[df_copy['EndOfDayQuote Open'] == 0, 'EndOfDayQuote PreviousClose']
    df_copy.loc[df_copy['EndOfDayQuote Open'] == 0, 'EndOfDayQuote Open'] =  df_copy.loc[df_copy['EndOfDayQuote Open'] == 0, 'EndOfDayQuote ExchangeOfficialClose']
    df_copy.loc[df_copy['EndOfDayQuote Open'] == 0, 'EndOfDayQuote Open'] =  1.0    
    #print(df_copy.sort_values(['EndOfDayQuote Open']))
    for target_code in tqdm(df_copy['Local Code'].unique()):
        ret = {}
        df = input_df.loc[df_copy.loc[:, 'Local Code']==target_code]
        ret['target'] = (df[end_label].shift(-6) - df[start_label].shift(-1)) / df[start_label].shift(-1)
        code_df = pd.DataFrame(ret, index=df.index)
        out_df.append(code_df)
    return pd.concat(out_df, axis=0)

In [ ]:
y = to_target(df_price)

In [ ]:
y[y.isin([np.inf, -np.inf])] = 0.0

# 欠損値の対応

移動平均線などの異なる行情報から作るので、新しくlocal codeが生まれた場合数日間はnanになる。  
直後の値で埋める

In [ ]:
new_X = X.groupby('Local Code').fillna(method='bfill')  # 過去の情報から作られる
new_X['Local Code'] = X['Local Code']
X = new_X

new_Y = pd.DataFrame([])
new_Y['Local Code'] = df_price['Local Code']
new_Y['target'] = y
new_Y = new_Y.groupby('Local Code').fillna(method='ffill')  # 未来の情報から作られる
y = new_Y['target']

# CVとModel

時系列データなのでCVはせずに訓練と評価とテスト期間に分割して実行する  
Todo: https://blog.amedama.jp/entry/time-series-cv time serise cv

In [ ]:
TRAIN_START = "2016-02-01"  # テクニカルを行うために１ヶ月のbuffer
TRAIN_END = "2019-12-30"
VAL_START = "2020-02-01"
VAL_END = "2020-11-30"

In [ ]:
# check number of data
print('train date', len(X.loc[TRAIN_START: TRAIN_END].index.unique()))
print('val date', len(X.loc[VAL_START: VAL_END].index.unique()))

In [ ]:
print(X.loc[TRAIN_START: TRAIN_END].isnull().sum())
print(X.loc[VAL_START: VAL_END].isnull().sum())
print(y.isnull().sum())

In [ ]:
def get_index(input_df, start, end):
    return input_df.reset_index().reset_index().set_index('EndOfDayQuote Date').loc[start:end, 'index'].values

idx_train = get_index(df_price, TRAIN_START, TRAIN_END)
idx_valid = get_index(df_price, VAL_START, VAL_END)
cv = [(idx_train, idx_valid)]

In [ ]:
def fit_lgbm(X, 
             y, 
             cv, 
             params: dict=None, 
             verbose: int=50):
    """lightGBM を CrossValidation の枠組みで学習を行なう function"""

    # パラメータがないときは、空の dict で置き換える
    if params is None:
        params = {}

    models = []
    # training data の target と同じだけのゼロ配列を用意
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    for i, (idx_train, idx_valid) in enumerate(cv): 
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)

        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        #print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')

    score = mean_squared_error(y, oof_pred) ** .5 / len(y)
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.10f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    # 目的関数. これの意味で最小となるようなパラメータを探します. 
    'objective': 'rmse', 

     # 学習率. 小さいほどなめらかな決定境界が作られて性能向上に繋がる場合が多いです、
    # がそれだけ木を作るため学習に時間がかかります
    'learning_rate': .1,

    # L2 Reguralization
    'reg_lambda': 1.,
    # こちらは L1 
    'reg_alpha': .1,

    # 木の深さ. 深い木を許容するほどより複雑な交互作用を考慮するようになります
    'max_depth': 5, 

    # 木の最大数. early_stopping という枠組みで木の数は制御されるようにしていますのでとても大きい値を指定しておきます.
     'n_estimators': 10000, 

    # 木を作る際に考慮する特徴量の割合. 1以下を指定すると特徴をランダムに欠落させます。小さくすることで, まんべんなく特徴を使うという効果があります.
    'colsample_bytree': .5, 

    # 最小分割でのデータ数. 小さいとより細かい粒度の分割方法を許容します.
    'min_child_samples': 10,

    # bagging の頻度と割合
    'subsample_freq': 3,
    'subsample': .9,

    # 特徴重要度計算のロジック(後述)
    'importance_type': 'gain', 
    'random_state': 71,
}


In [ ]:
oof_1, models_1 = fit_lgbm(X.values, y.values.reshape(-1), cv=cv, params=params, verbose=500)
models = {}
models['1'] = models_1

## validでx軸予測 vs y軸正解

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(oof_1[idx_valid], y.values.reshape(-1)[idx_valid])

In [ ]:
def visualize_importance(models, feat_train_df):
    """lightGBM の model 配列の feature importance を plot する
    CVごとのブレを boxen plot として表現します.

    args:
        models:
            List of lightGBM models
        feat_train_df:
            学習時に使った DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    ax.tick_params(axis='x', rotation=90)
    ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    return fig, ax

In [ ]:
fig, ax = visualize_importance(models['1'], X)

普通に株価が一番きいている  
ボリンジャーバンド  
7日移動平均線  
Local Codeの違いは微妙？？
予測が%でどれくらいに収まっているかチェック

# ポートフォリオ

In [ ]:
TEST_START = "2021-02-01" # 上昇トレンド (public board)
#TEST_START = "2021-01-25" # 下降トレンド
#TEST_START = "2021-03-01" # 下降トレンド
PRED_DATE = pd.Timestamp(TEST_START) - pd.Timedelta("3D")

In [ ]:
tmp_df = X.copy()
tmp_df['ret'] = (tmp_df['EndOfDayQuote ExchangeOfficialClose'].shift(-5))/(tmp_df['EndOfDayQuote Open'])-1
tmp_df.loc[TEST_START].sort_values(['ret']).tail(10)

## 予測に基づいてポートフォリを決める

In [ ]:
def pred_lgbm(X, y, models):
    scores = []
    preds = []
    length = len(X)
    for model in models:
        pred = model.predict(X)
        score = mean_squared_error(y, pred) ** .5 / length
        preds.append(pred)
        scores.append(score)        
        print('score', score)
    print('-' * 50)
    pred = np.vstack(preds).mean(axis=0)
    socre = np.vstack(scores).mean(axis=0)
    print('FINISHED | Whole RMSLE: {:.10f}'.format(score))
    return pred, score

In [ ]:
# load model
#path = '../submit 2/model/checkpoints.pickle'
#with open(path, 'rb') as f:
#    checkpoint = pickle.load(f)
#models = checkpoint['models']
#keys = checkpoint['keys']['1'][0]


#pred, score = pred_lgbm(X[keys].loc[PRED_DATE].values, y.loc[PRED_DATE].values, models['1'])
pred, score = pred_lgbm(X.loc[PRED_DATE].values, y.loc[PRED_DATE].values, models['1'])

pf_df = X.loc[PRED_DATE].copy()
pf_df['ret'] = pred
pf_df.sort_values(['ret'])

In [ ]:
a = pf_df[(pf_df['ret'] - pf_df['retrun1']).abs() >=0.01]
a[a['ret'] * a['retrun1'] >=0 ]

In [ ]:
top_k = 9
df_portfolio = pd.DataFrame()
df_portfolio['Local Code'] = pf_df.loc[PRED_DATE].sort_values(['ret']).tail(top_k)['Local Code']
df_portfolio['date'] = TEST_START
df_portfolio['budget'] = 1000000/top_k
df_portfolio[['date', 'Local Code', 'budget']]

In [ ]:
out_path = os.path.join('./', 'submit.csv')
df_portfolio[['date', 'Local Code', 'budget']].to_csv(out_path, index=False)

# backtest

In [ ]:
backtest_codes, backtest_price = Backtest.prepare_data('./data/')

In [ ]:
df_submit = Backtest.load_submit('./submit.csv')
df_results, df_stocks = Backtest.run(df_submit, backtest_codes, df_price)

In [ ]:
df_results

In [ ]:
df_stocks

In [ ]:
df_return = pd.DataFrame([])
for i in range(1, 6):
    df_return[f'day_{i}'] = (df_stocks[f'day_{i}']/df_stocks['entry'] -1)*100
df_return['Local Code'] = df_stocks['Local Code']
df_return = df_return.set_index('Local Code')
df_return.T.plot()

# analysis
top kの最適な数をチェック  
上昇相場と下降相場でシャープレシオがそこそこ高かったtop_k = 9を採用

In [ ]:
result = []
for top_k in list(range(5,21,1)):
    df_portfolio = pd.DataFrame()
    df_portfolio['Local Code'] = pf_df.loc[PRED_DATE].sort_values(['ret']).tail(top_k)['Local Code']
    df_portfolio['date'] = TEST_START
    df_portfolio['budget'] = 1000000/top_k
    df_portfolio[['date', 'Local Code', 'budget']]
    out_path = os.path.join('./', 'submit.csv')
    df_portfolio[['date', 'Local Code', 'budget']].to_csv(out_path, index=False)
    df_submit = Backtest.load_submit('./submit.csv')
    df_results, df_stocks = Backtest.run(df_submit, backtest_codes, df_price)
    df_results['top_k'] = top_k
    result.append(df_results)
pd.concat(result, axis=0)

# model save

In [ ]:
keys = {'1': [list(X.keys())]}
checkpoint = {'keys': keys, 'models': models}
import pickle
with open('./checkpoints.pickle', 'wb') as f:
      pickle.dump(checkpoint , f)